In [1]:
!pip install datasets

import numpy as np
from PIL import Image
from datasets import load_dataset

# Load the dataset with the `large_first_1k` subset
dataset = load_dataset('poloclub/diffusiondb', 'large_first_1k')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for poloclub/diffusiondb contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/poloclub/diffusiondb
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.


Generating train split: 0 examples [00:00, ? examples/s]

In [2]:
from transformers import pipeline
import tempfile

# Access the data
data = dataset['train']

from transformers import BlipProcessor, BlipForConditionalGeneration

def generate_caption_blip(image):
    blip_model_id = "Salesforce/blip-image-captioning-large"
    processor = BlipProcessor.from_pretrained(blip_model_id)
    model = BlipForConditionalGeneration.from_pretrained(blip_model_id)

    inputs = processor(images=image, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=30, num_beams=3)
    caption = processor.decode(outputs[0], skip_special_tokens=True)

    return caption

def generate_caption_pix2struct(image_url):
    pix2struct_pipeline = pipeline("image-to-text", model="google/pix2struct-textcaps-base")
    result = pix2struct_pipeline(image_url)
    return result[0]['generated_text'] if result else "Caption generation failed"

def generate_caption_gpt2(image_url):
    # Hypothetical pipeline for GPT-2 adapted for image-to-text
    gpt2_pipeline = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")
    result = gpt2_pipeline(image_url)
    return result[0]['generated_text'] if result else "Caption generation failed"


def save_temp_image(image):
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.png')
    image.save(temp_file, format='PNG')
    return temp_file.name  # Returns the file path


In [3]:
import random
import os

# Select a random sample from the dataset
sample = random.choice(dataset['train'])

# Generate a BLIP caption directly
blip_caption = generate_caption_blip(sample['image'])

# Save the image temporarily for Pix2Struct and GPT-2
image_path = save_temp_image(sample['image'])

# Generate captions using Pix2Struct and GPT-2
pix2struct_caption = generate_caption_pix2struct(image_path)
gpt2_caption = generate_caption_gpt2(image_path)

print(sample['prompt'])
print("BLIP Caption:", blip_caption)
print("Pix2Struct Caption:", pix2struct_caption)
print("GPT-2 Caption:", gpt2_caption)  # If applicable

# Cleanup temporary file
os.unlink(image_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


minecraft cosmic fantasy wallpaper synthwave mix title card, party rpg, 4 k 
BLIP Caption: minecraft is coming to the nintendo switch
Pix2Struct Caption: A man is holding a bottle of alcohol and the label Wheelheaert
GPT-2 Caption: a blurry photo of a cityscape with a sky background 


In [4]:
!pip install rouge-score
import os
import random
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer

num_samples = 10  # Number of samples to process

original_captions = []
blip_captions = []
pix2struct_captions = []
gpt2_captions = []

for _ in range(num_samples):
    sample = random.choice(dataset['train'])

    original_caption = sample['prompt']
    original_captions.append(original_caption)

    # BLIP caption
    blip_caption = generate_caption_blip(sample['image'])
    blip_captions.append(blip_caption)

    # Temporarily save the image for Pix2Struct and GPT-2
    image_path = save_temp_image(sample['image'])

    # Pix2Struct caption
    pix2struct_caption = generate_caption_pix2struct(image_path)
    pix2struct_captions.append(pix2struct_caption)

    # GPT-2 caption
    gpt2_caption = generate_caption_gpt2(image_path)
    gpt2_captions.append(gpt2_caption)

    print(original_caption)
    print(blip_caption)
    print(pix2struct_caption)
    print(gpt2_caption)



    # Cleanup temporary file
    os.unlink(image_path)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=99505ec02395ce7120ab8f629c9c9670b40194d2bd13e948757fde1140ce89a3
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


building on fire, golden hour, ultrahd, ultra sharp, hyper detailed, 8 k, shot on 3 5 mm film camera, tumblr aesthetic, wallpaper 
firefighters are battling to put out a fire in an apartment building
A red and white building with a red and white sign that says "The House".
a fire hydrant is shown in front of a building 


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


a photograph of a futuristic street scene, brutalist style architecture, perfectly straight edges, shop signs, advertising, finely detailed oil painting, impasto brush strokes, soft light, 8 k, dramatic composition, dramatic lighting, sharp focus, octane render, masterpiece, by adrian ghenie and jenny saville and zhang jingna 
painting of a group of people walking down a street next to a tall building
A man is standing in front of a building that says "The Old Man's House"
people walking on a sidewalk near a building 


...

a sexy seductive portrait of lady gaga wearing transparent pink latex with soft rembrandt lighting, hajime sorayama 
there is a dog that is laying down with a pink object in it's mouth
A person is holding a bottle of wine that says 'a bottle of wine'.
a woman is holding a pink flower in her hand 


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


a painting by hiroshi nagai of magnificent golden chinese palaces, lotus garden, pine tree. hq scan. 
there is a painting of a building in the middle of a lake
A boat with a sailboat on the water
a large red and white clock tower in the middle of a lake 


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


a beautiful wide - lens landscape photograph of a bioluminescent mushroom jungle, natural lighting, shot on a pentax k 1 mark ii 
there are many plants that are lit up in the dark
A man is holding a book titled The Secret Life of the Mind.
a forest filled with trees and shrubs 


In [7]:
import nltk
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu
from nltk.translate.meteor_score import single_meteor_score
from rouge_score import rouge_scorer

nltk.download('wordnet')

# Initialize the ROUGE scorer with additional 'rouge2'
rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeLsum'], use_stemmer=True)

# Function to calculate scores
def calculate_scores(original_captions, generated_captions):
    # BLEU scores
    references = [[caption.split()] for caption in original_captions]
    candidates = [caption.split() for caption in generated_captions]
    bleu_score = corpus_bleu(references, candidates, smoothing_function=SmoothingFunction().method1)

    # METEOR scores (tokenized inputs directly used)
    meteor_scores = [single_meteor_score(ref[0], can) for ref, can in zip(references, candidates)]
    avg_meteor_score = sum(meteor_scores) / len(meteor_scores)

    # ROUGE scores
    rouge1_scores = []
    rouge2_scores = []
    rougeLsum_scores = []
    for ref, gen in zip(original_captions, generated_captions):
        score = rouge_scorer.score(ref, gen)
        rouge1_scores.append(score['rouge1'].fmeasure)
        rouge2_scores.append(score['rouge2'].fmeasure)
        rougeLsum_scores.append(score['rougeLsum'].fmeasure)
    avg_rouge1_score = sum(rouge1_scores) / len(rouge1_scores)
    avg_rouge2_score = sum(rouge2_scores) / len(rouge2_scores)
    avg_rougeLsum_score = sum(rougeLsum_scores) / len(rougeLsum_scores)

    return bleu_score, avg_meteor_score, avg_rouge1_score, avg_rouge2_score, avg_rougeLsum_score

# Calculate scores for each model
results_blip = calculate_scores(original_captions, blip_captions)
results_pix2struct = calculate_scores(original_captions, pix2struct_captions)
results_gpt2 = calculate_scores(original_captions, gpt2_captions)

# Print the scores
print(f"BLIP Scores - BLEU: {results_blip[0]}, METEOR: {results_blip[1]}, ROUGE1f: {results_blip[2]}, ROUGE2f: {results_blip[3]}, ROUGELf: {results_blip[4]}")
print(f"Pix2Struct Scores - BLEU: {results_pix2struct[0]}, METEOR: {results_pix2struct[1]}, ROUGE1f: {results_pix2struct[2]}, ROUGE2f: {results_pix2struct[3]}, ROUGELf: {results_pix2struct[4]}")
print(f"GPT-2 Scores - BLEU: {results_gpt2[0]}, METEOR: {results_gpt2[1]}, ROUGE1f: {results_gpt2[2]}, ROUGE2f: {results_gpt2[3]}, ROUGELf: {results_gpt2[4]}")

BLIP Scores - BLEU: 0.002976092891994063, METEOR: 0.042654581571152074, ROUGE1f: 0.12732041287734028, ROUGE2f: 0.017998540412333515, ROUGELf: 0.10102962778860122
Pix2Struct Scores - BLEU: 0.0019313149419802492, METEOR: 0.03666972308874494, ROUGE1f: 0.08902202140163142, ROUGE2f: 0.003389830508474576, ROUGELf: 0.08313966846045494
GPT-2 Scores - BLEU: 0.0014137737006247426, METEOR: 0.033843046606143616, ROUGE1f: 0.10148337090269426, ROUGE2f: 0.01, ROUGELf: 0.09523337090269426


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
